<a href="https://colab.research.google.com/github/GiulioAppetito/Progetto_MOBD_2122/blob/main/prova1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/ProgettoMOBD

/content/drive/My Drive/ProgettoMOBD


In [3]:
import sklearn
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn_pandas import CategoricalImputer
from sklearn.ensemble import IsolationForest, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
from sklearn.svm import SVC, LinearSVC
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn import tree
import graphviz
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

np.random.seed = 123

In [ ]:
# LETTURA DATASET
data = pd.read_csv("train.csv", sep=",")
print(data.head())
# print("\nLabels: {}".format(np.unique(data.Y, return_counts=True)))

# print("\n# of NaNs values for each column:")
# print(data.isnull().sum(axis=0))

x = data.iloc[:, :-1].to_numpy()
y = data.iloc[:, -1].to_numpy()
# print(x.shape)
# print(y.shape)

# stratify: to mantain the same mean as y for y_tr and y_ts
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.3, random_state=258, stratify=y)

# print(x_tr.shape)
# print(y_tr.shape)
# print(x_ts.shape)
# print(y_ts.shape)

# print(np.unique(y, return_counts=True)[1]/len(y))
# print(np.unique(y_tr, return_counts=True)[1]/len(y_tr))
# print(np.unique(y_ts, return_counts=True)[1]/len(y_ts))

In [9]:
# valori nan
imp = SimpleImputer(missing_values=np.nan, strategy = 'most_frequent')
x_tr_notnan = imp.fit_transform(x_tr)
x_ts_notnan = imp.transform(x_ts)

#gestione features categoriche
#dato che però questi sono ndarrays, per fare la one hot coded technique dobbiamo usare dataframes

x_tr_df = pd.DataFrame(x_tr_notnan)
x_ts_df = pd.DataFrame(x_ts_notnan)

x_tr_df.columns=['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13']
x_ts_df.columns=['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13']

print(x_tr_df.head())
print('*******************')
#print(x_ts_df.head())

# one hot code

encoder = OneHotEncoder(handle_unknown='ignore')
encoded_df = pd.DataFrame(encoder.fit_transform(x_tr_df[['F1']]).toarray())
#print(encoded_df.head())

print('*******************')
final_df = x_tr_df.join(encoded_df)
final_df.rename(columns={0:'K1',1:'K2',2:'K3',3:'K4',4:'K5',5:'K6',6:'K7',7:'K8'}, inplace=True)
final_df.drop('F1', axis=1, inplace=True)
print(final_df.head())


encoded_df = pd.DataFrame(encoder.fit_transform(x_tr_df[['F7']]).toarray())
#print(encoded_df.head())

print('*******************')
final_df = final_df.join(encoded_df)
final_df.rename(columns={0:'M1',1:'M2',2:'M3',3:'M4',4:'M5',5:'M6'}, inplace=True)
final_df.drop('F7', axis=1, inplace=True)
print(final_df.head())

# encoder = OneHotEncoder(handle_unknown='ignore')
#encoded_df = pd.DataFrame(encoder.fit_transform(final_df[['F3']]).toarray())
#print(encoded_df.head())

#print('*******************')
#final_df = final_df.join(encoded_df)
#final_df.rename(columns={0:'R1',1:'R2',2:'R3',3:'R4',4:'R5',5:'R6',6:'R7',7:'R8',8:'R9',9:'R10',10:'R11',11:'R12',12:'R13',13:'R14',14:'R15',15:'R16'}, inplace=True)
#final_df.drop('F3', axis=1, inplace=True)
#print(final_df.head())


   F0  F1      F2  F3  F4  F5   F6  F7         F8      F9 F10 F11 F12  F13
0  27  K3  130067  R6  10  Q1   P2  M1  caucasian  Female   0   0  65  USA
1  18  K3  703067  R3   7  Q1   P6  M4  caucasian    Male   0   0  20  USA
2  33  K6  144949  R1  13  Q2   P6  M2  caucasian    Male   0   0  65  USA
3  54  K3  176681  R2   9  Q2   P1  M2      black    Male   0   0  20  USA
4  39  K3  115076  R6  10  Q2  P10  M2  caucasian    Male   0   0  40  USA
*******************
*******************
   F0      F2  F3  F4  F5   F6  F7         F8      F9 F10  ... F12  F13   K1  \
0  27  130067  R6  10  Q1   P2  M1  caucasian  Female   0  ...  65  USA  0.0   
1  18  703067  R3   7  Q1   P6  M4  caucasian    Male   0  ...  20  USA  0.0   
2  33  144949  R1  13  Q2   P6  M2  caucasian    Male   0  ...  65  USA  0.0   
3  54  176681  R2   9  Q2   P1  M2      black    Male   0  ...  20  USA  0.0   
4  39  115076  R6  10  Q2  P10  M2  caucasian    Male   0  ...  40  USA  0.0   

    K2   K3   K4   K5   K6   